# Data Collection with an API using Python

BY MAUZUM SHAMIL

write code to authenticate with the Spotify API and obtain an access token using the Client Credentials Flow

In [1]:
import requests
import base64

# replace with your own client id and client secret
CLIENT_ID = '391f9be5e17342259f74de4233####'
CLIENT_SECRET = '85d14b4c625c4482ab0cba2f74######'

# Base64 encode the client id and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [2]:
%pip install spotipy


  Obtaining dependency information for spotipy from https://files.pythonhosted.org/packages/b8/e8/4c099f9431ec9a86f576b344702cd4446d1ff7df09b172dc1951f25d58b1/spotipy-2.23.0-py3-none-any.whl.metadata
  Obtaining dependency information for redis>=3.5.3 from https://files.pythonhosted.org/packages/65/f2/540ad07910732733138beb192991c67c69e7f6ebf549ce1a3a77846cbae7/redis-5.0.4-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/252.0 kB ? eta -:--:--
   ----------- ---------------------------- 71.7/252.0 kB 1.3 MB/s eta 0:00:01
   --------------------------- ------------ 174.1/252.0 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 252.0/252.0 kB 1.9 MB/s eta 0:00:00



# write a Python function to extract detailed information about each track in any Spotify playlist

In [3]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # set up spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # add more attributes as needed (go through the documentation to know what more you can add)
        }

        music_data.append(track_data)

    # create a pandas dataframe from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

# Now give the playlist link id to get the information using the function 

In [4]:
#  add the playlist id of any playlist on Spotify here
playlist_id = '1gfWsOG1WAoxNeUMMktZbq'

# call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

print(music_df)

                                     Track Name  \
0                                 Bijlee Bijlee   
1                                   Expert Jatt   
2   Kaun Nachdi (From "Sonu Ke Titu Ki Sweety")   
3                                   Na Na Na Na   
4                                   Patiala Peg   
..                                          ...   
95                                                
96                               Move Your Lakk   
97                    Patola (From "Blackmail")   
98            Ban Ja Rani (From "Tumhari Sulu")   
99          Hauli Hauli (From "De De Pyaar De")   

                                    Artists  \
0                             Harrdy Sandhu   
1                                     Nawab   
2                Guru Randhawa, Neeti Mohan   
3                                    J Star   
4                            Diljit Dosanjh   
..                                      ...   
95                                            
96  Diljit 

To get the playlist ID of any other playlist on Spotify, just copy the link of the playlist

In [5]:
music_df.to_csv("musicdata.csv")

In [6]:
music_df

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Bijlee Bijlee,Harrdy Sandhu,Bijlee Bijlee,3tG0IGB24sRhGFLs5F1Km8,1iZLpuGMr4tn1F5bZu32Kb,69,2021-10-30,168450,False,https://open.spotify.com/track/1iZLpuGMr4tn1F5...,...,0.670,1,-5.313,0,0.1430,0.26900,0.000000,0.0733,0.643,100.004
1,Expert Jatt,Nawab,Expert Jatt,2gibg5SCTep0wsIMefGzkd,7rr6n1NFIcQXCsi43P0YNl,65,2018-01-18,199535,False,https://open.spotify.com/track/7rr6n1NFIcQXCsi...,...,0.948,6,-2.816,0,0.1990,0.29800,0.000000,0.0784,0.647,172.038
2,"Kaun Nachdi (From ""Sonu Ke Titu Ki Sweety"")","Guru Randhawa, Neeti Mohan",High Rated Gabru - Guru Randhawa,6EDbwGsQNQRLf73c7QwZ2f,3s7m0jmCXGcM8tmlvjCvAa,63,2019-03-02,183373,False,https://open.spotify.com/track/3s7m0jmCXGcM8tm...,...,0.830,4,-3.981,0,0.0455,0.03570,0.000000,0.0419,0.753,127.999
3,Na Na Na Na,J Star,Na Na Na Na,4xBqgoiRSOMU1VlKuntVQW,5GjxbFTZAMhrVfVrNrrwrG,52,2015,209730,False,https://open.spotify.com/track/5GjxbFTZAMhrVfV...,...,0.863,3,-3.760,1,0.0413,0.37600,0.000014,0.0916,0.807,95.000
4,Patiala Peg,Diljit Dosanjh,Do Gabru - Diljit Dosanjh & Akhil,1uxDllRe9CPhdr8rhz2QCZ,6TikcWOLRsPq66GBx2jk67,48,2018-07-10,188314,False,https://open.spotify.com/track/6TikcWOLRsPq66G...,...,0.811,5,-3.253,0,0.1840,0.02590,0.000000,0.3110,0.835,175.910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,,,,2jw92hf4mnISbYywvU3Anj,3OZr3vo7SmYpn5XqeQEAOM,0,0000,203207,False,https://open.spotify.com/track/3OZr3vo7SmYpn5X...,...,0.842,6,-4.109,1,0.0745,0.00814,0.000013,0.2120,0.915,156.051
96,Move Your Lakk,"Diljit Dosanjh, Badshah, Sonakshi Sinha",Move Your Lakk,0V06TMGQQQkvKxNmFlKyEj,3aYMKdSitJeHUCZO8Wt6fw,51,2017-03-29,194568,False,https://open.spotify.com/track/3aYMKdSitJeHUCZ...,...,0.816,2,-5.595,1,0.1480,0.03790,0.000153,0.1230,0.744,99.992
97,"Patola (From ""Blackmail"")","Guru Randhawa, Preet Hundal","Patola (From ""Blackmail"")",2XAAIDEpPb57NsKgAHLGVQ,17LZzRCY0iFWlDDuAG7BlM,56,2018-03-05,184410,False,https://open.spotify.com/track/17LZzRCY0iFWlDD...,...,0.901,3,-2.719,0,0.0508,0.12600,0.000000,0.0692,0.914,87.998
98,"Ban Ja Rani (From ""Tumhari Sulu"")",Guru Randhawa,High Rated Gabru - Guru Randhawa,6EDbwGsQNQRLf73c7QwZ2f,7cQtGVoPCK9DlspeYjdHOA,59,2019-03-02,225938,False,https://open.spotify.com/track/7cQtGVoPCK9Dlsp...,...,0.692,9,-4.718,0,0.0577,0.20600,0.000000,0.1240,0.487,102.015


## Summary
So, this is how you can collect data from an API using Python. Using an API for data collection is a powerful way to obtain real-time or historical data for data analysis, machine learning models, or any data-driven application.

#  ANOTHER PLAYLIST

In [10]:
# add the playlist id of any playlist on Spotify here
playlist_id2 = '1gfWsOG1WAoxNeUMMktZbq'

# call the function to get the music data from the playlist and store it in a DataFrame
music_df2 = get_trending_playlist_data(playlist_id2, access_token)

print(music_df2)

                                     Track Name  \
0                                 Bijlee Bijlee   
1                                   Expert Jatt   
2   Kaun Nachdi (From "Sonu Ke Titu Ki Sweety")   
3                                   Na Na Na Na   
4                                   Patiala Peg   
..                                          ...   
95                                                
96                               Move Your Lakk   
97                    Patola (From "Blackmail")   
98            Ban Ja Rani (From "Tumhari Sulu")   
99          Hauli Hauli (From "De De Pyaar De")   

                                    Artists  \
0                             Harrdy Sandhu   
1                                     Nawab   
2                Guru Randhawa, Neeti Mohan   
3                                    J Star   
4                            Diljit Dosanjh   
..                                      ...   
95                                            
96  Diljit 

In [11]:
music_df2.to_csv("musicdata2.csv")

In [12]:
music_df2

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Bijlee Bijlee,Harrdy Sandhu,Bijlee Bijlee,3tG0IGB24sRhGFLs5F1Km8,1iZLpuGMr4tn1F5bZu32Kb,69,2021-10-30,168450,False,https://open.spotify.com/track/1iZLpuGMr4tn1F5...,...,0.670,1,-5.313,0,0.1430,0.26900,0.000000,0.0733,0.643,100.004
1,Expert Jatt,Nawab,Expert Jatt,2gibg5SCTep0wsIMefGzkd,7rr6n1NFIcQXCsi43P0YNl,65,2018-01-18,199535,False,https://open.spotify.com/track/7rr6n1NFIcQXCsi...,...,0.948,6,-2.816,0,0.1990,0.29800,0.000000,0.0784,0.647,172.038
2,"Kaun Nachdi (From ""Sonu Ke Titu Ki Sweety"")","Guru Randhawa, Neeti Mohan",High Rated Gabru - Guru Randhawa,6EDbwGsQNQRLf73c7QwZ2f,3s7m0jmCXGcM8tmlvjCvAa,63,2019-03-02,183373,False,https://open.spotify.com/track/3s7m0jmCXGcM8tm...,...,0.830,4,-3.981,0,0.0455,0.03570,0.000000,0.0419,0.753,127.999
3,Na Na Na Na,J Star,Na Na Na Na,4xBqgoiRSOMU1VlKuntVQW,5GjxbFTZAMhrVfVrNrrwrG,52,2015,209730,False,https://open.spotify.com/track/5GjxbFTZAMhrVfV...,...,0.863,3,-3.760,1,0.0413,0.37600,0.000014,0.0916,0.807,95.000
4,Patiala Peg,Diljit Dosanjh,Do Gabru - Diljit Dosanjh & Akhil,1uxDllRe9CPhdr8rhz2QCZ,6TikcWOLRsPq66GBx2jk67,48,2018-07-10,188314,False,https://open.spotify.com/track/6TikcWOLRsPq66G...,...,0.811,5,-3.253,0,0.1840,0.02590,0.000000,0.3110,0.835,175.910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,,,,2jw92hf4mnISbYywvU3Anj,3OZr3vo7SmYpn5XqeQEAOM,0,0000,203207,False,https://open.spotify.com/track/3OZr3vo7SmYpn5X...,...,0.842,6,-4.109,1,0.0745,0.00814,0.000013,0.2120,0.915,156.051
96,Move Your Lakk,"Diljit Dosanjh, Badshah, Sonakshi Sinha",Move Your Lakk,0V06TMGQQQkvKxNmFlKyEj,3aYMKdSitJeHUCZO8Wt6fw,51,2017-03-29,194568,False,https://open.spotify.com/track/3aYMKdSitJeHUCZ...,...,0.816,2,-5.595,1,0.1480,0.03790,0.000153,0.1230,0.744,99.992
97,"Patola (From ""Blackmail"")","Guru Randhawa, Preet Hundal","Patola (From ""Blackmail"")",2XAAIDEpPb57NsKgAHLGVQ,17LZzRCY0iFWlDDuAG7BlM,56,2018-03-05,184410,False,https://open.spotify.com/track/17LZzRCY0iFWlDD...,...,0.901,3,-2.719,0,0.0508,0.12600,0.000000,0.0692,0.914,87.998
98,"Ban Ja Rani (From ""Tumhari Sulu"")",Guru Randhawa,High Rated Gabru - Guru Randhawa,6EDbwGsQNQRLf73c7QwZ2f,7cQtGVoPCK9DlspeYjdHOA,59,2019-03-02,225938,False,https://open.spotify.com/track/7cQtGVoPCK9Dlsp...,...,0.692,9,-4.718,0,0.0577,0.20600,0.000000,0.1240,0.487,102.015
